In [ ]:
from dotenv import get_key
from pymongo import MongoClient
import pymongo
import psycopg2 # Remover
from pprint import pprint  # Imprimir bsons formatados
from utils import *

STRING = get_key('.env', 'MG_STRING')
NAME = get_key('.env', 'PG_NAME')
USER = get_key('.env', 'PG_USER')
PASSWORD = get_key('.env', 'PG_PASSWORD')
HOST = get_key('.env', 'PG_HOST')
PORT = get_key('.env', 'PG_PORT')

## Funções para acesso ao MongoDB

In [ ]:
def create_mongo_connection():
	"""Cria uma conexão com o banco de dados MongoDB. Retorna o cliente."""
	try:
		client = MongoClient(STRING, server_api=pymongo.server_api.ServerApi(version="1"))
		client.admin.command("ping")
		print("---- Conexão bem sucedida ----")
	except Exception as e:
		raise Exception(
			"The following error occurred: ", e)
	return client

def end_mongo_connection(client):
	client.close()
	print("---- Conexão encerrada -----")

## Criação de tabelas no banco

In [48]:
def create_tables(db, col):
	match col:
		case 'pais':
			db.command({
				"collMod": "pais",
				"validator": {
					"$jsonSchema": {
						"bsonType": "object",
						"properties": {
							"nome": {"bsonType": "string"},
							"codigo": {"bsonType": "string"},
							"continente": {"bsonType": "string"},
							"grupos": {
								"bsonType": "array",
								"items": {
									"bsonType": "object",
									"properties": {
										"sigla": {"bsonType": "string"}
									}
								}
							},
							"_id": {"bsonType": "objectId"}
						},
						"title": "pais"
					}
				},
				'validationLevel': 'moderate',
				'validationAction': 'warn'
			})
		case 'emissaoGas':
			collection_name = 'emissaoGas'
		case _:
			raise ValueError(f"Nome de coleção inválido: {col}")

In [ ]:
def consultas(client):
	# for db_info in client.list_database_names():
	# 	print(db_info)
	# db = client["Biblioteca"]
	# # collections = db.list_collection_names()
	# # print("Coleções em samples_mflix:", collections)
	# result = db.books.find({"authors": "Machado de Assis"}, {'_id': 0, 'title': 1})
	
	# for doc in result:
	# 	pprint(doc)

	db = client["projeto_mc536"]
	# colecao = db["teste"]
	# # db.teste.insertOne({'name': 'José',
	# # 				 'age': 58})
	# colecao.insert_one({'campo': 'valor exemplo'})
	# result = db['teste'].find({}, {'_id':0})
	# for doc in result:
	# 	pprint(doc)


	# result = db['teste'].find({}, {'_id':0})
	# print('---')
	# for doc in result:
	# 	pprint(doc)

	# Corrigindo a criação da coleção 'pais' caso ela não exista antes de aplicar o validator

	query = 'SELECT * FROM PAIS;'
	conn = create_pg_connection()
	cursor = conn.cursor()
	cursor.execute(query)
	rows = cursor.fetchall()
	for row in rows:
		print(row)

	'''pais {
	sigla:
	nome:
	continente:
	grupos: String[]
	estatisticas: {
		ano:
		pib:
		geografia: {
			area_tipo:
			num_habitantes:
			area_km2:
		} []
	} []
}'''

	entries = {'pais': [
			{
				'continente': 18,
				'codigo': 'HW',
				'nome': 'Hogwarts',
				'grupos': [{'sigla': 'MAGIA'}],
				'preco' : 15
			}
		],
		'emissaoGas': [
			{
				'paisCodigo': 'BRA',
				'ano': 2005,
				'emissoes': [{
					'nome': 'Dióxido de Carbono',
					'formula': 'CO2',
					'valor': 320
				}]
			},
			{
				'paisCodigo': 'USA',
				'ano': 1998,
				'emissoes': [{
					'nome': 'Metano',
					'formula': 'CH4',
					'valor': 1000
				}]
			}
		]}
	
	for col in entries.keys():

		if col not in db.list_collection_names():
			db.create_collection(col)

		create_tables(db, col)
		db[col].delete_many({})
		db[col].insert_many(entries[col])

		print(f'> Coleção {col} <')
		# Exibindo os documentos inseridos
		for doc in db[col].find({}, {'_id': 0}):
			pprint(doc)
		
client = create_mongo_connection()
consultas(client)
end_mongo_connection(client)


In [ ]:
client = create_mongo_connection()
consultas(client)
end_mongo_connection(client)

In [ ]:
query = """
SELECT * FROM PAIS;
"""

conn = create_pg_connection()
cursor = conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
	print(row[1])
end_pg_connection(conn)

In [ ]:
query = 'SELECT * FROM PAIS;'
conn = create_pg_connection()
cursor = conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()
paises = []
for row in rows:
	paises.append({
					'sigla': row[0],
					'nome': row[1],
					'continente': row[2],
					'estatisticas': []
				})
# Grupos
for pais in paises:
	query = f'''SELECT SIGLA
	FROM
	GRUPO G INNER JOIN PAIS_GRUPO PG
		ON G.ID_GRUPO = PG.GRUPO_ID_GRUPO
	WHERE PAIS_CODIGO = '{pais['sigla']}';'''
	cursor.execute(query)
	rows = cursor.fetchall()
	grupos = [row[0] for row in rows]
	if len(grupos) > 0:
		pais['grupos'] = grupos

	query = f'''SELECT ANO,
	VALOR_USD
	FROM PIB
	WHERE CODIGO_PAIS = '{pais['sigla']}'
	ORDER BY ANO;'''
	cursor.execute(query)
	rows = cursor.fetchall()
	for row in rows:
		est = {
			'ano': row[0],
			'pib': row[1]
		}
		pais['estatisticas'].append(est)



pprint(paises)

'''pais {
x	sigla:
x	nome:
x	continente:
x	grupos: String[]
estatisticas: {
x	ano:
x	pib:
	geografia: {
		area_tipo:
		num_habitantes:
		area_km2:
	} []
} []
}'''

Nome: projeto_mc536
[{'continente': 'Asia',
  'estatisticas': [{'ano': 2000, 'pib': 6206547590.074026},
                   {'ano': 2001, 'pib': 5621147630.730684},
                   {'ano': 2002, 'pib': 7228795918.925245},
                   {'ano': 2003, 'pib': 7867263256.28817},
                   {'ano': 2004, 'pib': 7978515641.583037},
                   {'ano': 2005, 'pib': 8874480195.583212},
                   {'ano': 2006, 'pib': 9349921885.778885},
                   {'ano': 2007, 'pib': 10642671963.11354},
                   {'ano': 2008, 'pib': 11060395115.680857},
                   {'ano': 2009, 'pib': 13426272073.23976},
                   {'ano': 2010, 'pib': 15354612541.505938},
                   {'ano': 2011, 'pib': 15420077665.532305},
                   {'ano': 2012, 'pib': 17386490239.353626},
                   {'ano': 2013, 'pib': 18360263162.757835},
                   {'ano': 2014, 'pib': 18860496494.04041},
                   {'ano': 2015, 'pib': 19134221644.

'pais {\nx\tsigla:\nx\tnome:\nx\tcontinente:\nx\tgrupos: String[]\nestatisticas: {\n\tano:\n\tpib:\n\tgeografia: {\n\t\tarea_tipo:\n\t\tnum_habitantes:\n\t\tarea_km2:\n\t} []\n} []\n}'